# Developing With RDDs:  Laziness

In this lab, you will familarize with Spark's core abstraction, the `RDD`, and how it defers data processing until absolutely necessary.

## Objectives

1. Use this Notebook to inerarct with Spark
2. Use the `SparkContext` to get an `RDD`.
3. Use the dataset to familiarize yourself with its basic capabilities, including deferred data processing.


## Instructions

In order to work with data using Spark, we need to get our hands on a dataset.  Spark calls such a dataset a "resilient distributed dataset", or `RDD`.  There are several ways to get your hands on an `RDD`.  In most cases, however, you're going to use a `SparkContext` to do that.

Let's start with something simple:  a plain, ol' Scala collection of the integers from 1 to 10,000.


In [ ]:
val numbers = 1 to 10000

Ok.  We now have our plain, ol' Scala collection.  Next, take a look at your shell.  Did you notice the rather unassuming message `Spark context available as sc.`?  That means that the Spark shell has already created a `SparkContext` instance for you and has bound it to the variable `sc`.  Let's use it to turn our Scala collection into a Spark `RDD`.


In [ ]:
val rdd = sc.parallelize(numbers)

That didn't hurt at all, did it?  Ok, we are now ready to slice & dice like a true [data scientist](https://hbr.org/2012/10/data-scientist-the-sexiest-job-of-the-21st-century/).  What are some of the things we might want to do with such a dataset?  Well, let's *see* what we can do.  If you type `rdd.` in the shell, then hit `TAB`, you'll get a list of the methods available to you:

Did you see a long list of function? Something like this:

```scala
++                         aggregate                  asInstanceOf               cache
cartesian                  checkpoint                 coalesce                   collect
compute                    context                    count                      countApprox
countApproxDistinct        countByValue               countByValueApprox         dependencies
distinct                   filter                     filterWith                 first
flatMap                    flatMapWith                fold                       foreach
foreachPartition           foreachWith                getCheckpointFile          getStorageLevel
glom                       groupBy                    id                         intersection
isCheckpointed             isEmpty                    isInstanceOf               iterator
keyBy                      localCheckpoint            map                        mapPartitions
mapPartitionsWithContext   mapPartitionsWithIndex     mapPartitionsWithSplit     mapWith
max                        min                        name                       name_=
partitioner                partitions                 persist                    pipe
preferredLocations         randomSplit                reduce                     repartition
sample                     saveAsObjectFile           saveAsTextFile             setName
sortBy                     sparkContext               subtract                   take
takeOrdered                takeSample                 toArray                    toDebugString
toJavaRDD                  toLocalIterator            toString                   top
treeAggregate              treeReduce                 union                      unpersist
zip                        zipPartitions              zipWithIndex               zipWithUniqueId
```

Take a moment to look at all of the crazy things we can do.  There are many, but first things `first`. What would you expect if we wrote something like `rdd.first`? Well, let's try it...

In [ ]:
rdd.first

Not surprisingly, first returns the first item in the RDD. 

Let's try a few more things. What if you want the first n items?  Easy.

In [ ]:
rdd.take(5)

We can also find out all sorts of other things, like the total number of items or the sum of the elements.

In [ ]:
rdd.count

In [ ]:
rdd.sum

### Transformations

All of the operations we've performed so far are called "actions", because they cause Spark to actually process the data.  Another set of methods on `RDD`s is called "transformations", because they don't cause any data processing to happen.  This is one of the key design features of Spark:  it's as lazy as it can be, deferring calculation until it's absolutely required.  What are some of these transformations?  Let's have a look, starting with one of the most basic, `map`.

In [ ]:
val strings = rdd.map("#" + _)

We just told Spark that we want to convert the `RDD` into a collection of `String`s from its current form of `Int`s.  Now, we haven't actually converted them yet, because we haven't invoked any of the action methods.  This is important, because Spark can save a lot of processing by examining the transformations and only processing the necessary items.  Let's use `first` again to see what happens with this transformation.

In [ ]:
strings.first

What's important here is that Spark didn't convert all 10,000 entries into strings with a "#" in front of them, then return the first one.  It recognized that we only needed the first item, so only converted the first one to a String and returned it.  How do we know?  Let's time it, then compare to what would happen if we forced Spark to convert all of the entries to Strings, then return the first one.

In [ ]:
val start = System.currentTimeMillis; 
println(rdd.map("#" + _).first); 
println("took " + (System.currentTimeMillis - start) + " ms")

Ok.  On my machine that took about 25 ms.
Now, let's force all elements to be converted before taking the first.

In [ ]:
val start = System.currentTimeMillis; 
println(rdd.map("#" + _).collect.head); 
println("took " + (System.currentTimeMillis - start) + " ms")

Notice how we called `collect` this time, which forced Spark to convert each item, return the collection of Strings to us, then head to return the first item from the collection.  This time it took around 52 ms, which is not a huge difference.  Let's start seeing if we can exaggerate the results by iterating, increasing the size of the original set of numbers each time.  We used 10,000 before; let's go to 100,000.

In [ ]:
val n = 100000;
val numbers = sc.parallelize(1 to n)
val start = System.currentTimeMillis; 
println(numbers.map("#" + _).first); 
println("took " + (System.currentTimeMillis - start) + " ms")

In [ ]:
val start = System.currentTimeMillis; 
println(numbers.map("#" + _).collect.head); 
println("took " + (System.currentTimeMillis - start) + " ms")

Hmm.  In my case I got 12 ms versus 45 ms.  That's not much different.  What do you get on your machine?  Iterate again, using 1,000,000.

In [ ]:
val n = 1000000
val numbers = sc.parallelize(1 to n)
val start = System.currentTimeMillis; 
println(numbers.map("#" + _).first); 
println("took " + (System.currentTimeMillis - start) + " ms")

In [ ]:
val start = System.currentTimeMillis
println(numbers.map("#" + _).collect.head)
println("took " + (System.currentTimeMillis - start) + " ms")

Now we're starting to see a difference!  Let's go to 5,000,000.

In [ ]:
val n = 5000000
val numbers = sc.parallelize(1 to n)
val start = System.currentTimeMillis; 
println(numbers.map("#" + _).first); 
println("took " + (System.currentTimeMillis - start) + " ms")

In [ ]:
val start = System.currentTimeMillis; 
println(numbers.map("#" + _).collect.head); 
println("took " + (System.currentTimeMillis - start) + " ms")

In my case, I got 22 ms vs. 9003 ms.

Holy huge difference, Batman!  What kind of differences are you seeing on your machine?  Notice how, despite the increase in the size of the collection, the time it took using `first` remained fairly constant.  Spark is clearly being smart about how much work really needs to be done, only transforming and returning a single element instead of the whole one!


### Conclusion

What we've seen in this lab is how Spark is deferring execution until it's absolutely required, and only processing the data that must be processed in order to achieve the desired result.